# Regras de associação

In [1]:
%pip install apyori pandas &> /dev/null


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from apyori import apriori


In [4]:
base= pd.read_csv('./data.csv', sep=';', encoding='utf-8').map(lambda bool_str: bool_str == "Sim")
base


,Leite,Café,Cerveja,Pão,Manteiga,Arroz,Feijão
0,False,True,False,True,True,False,False
1,True,False,True,True,True,False,False
2,False,True,False,True,True,False,False
3,True,True,False,True,True,False,False
4,False,False,True,False,False,False,False
5,False,False,False,False,True,False,False
6,False,False,False,True,False,False,False
7,False,False,False,False,False,False,True
8,False,False,False,False,False,True,True
9,False,False,False,False,False,True,False


In [5]:
base.shape


(10, 7)

Transformar o dataframe em uma lista de transações.

In [12]:
txs = []
for i in range(len(base)):
  l = []
  cols_len = len(base.columns)
  for j, col_name in enumerate(base.columns):
    cell_val = base.values[i, j]
    if cell_val: l.append(col_name)
  txs.append(l)

txs


[['Café', 'Pão', 'Manteiga'],
 ['Leite', 'Cerveja', 'Pão', 'Manteiga'],
 ['Café', 'Pão', 'Manteiga'],
 ['Leite', 'Café', 'Pão', 'Manteiga'],
 ['Cerveja'],
 ['Manteiga'],
 ['Pão'],
 ['Feijão'],
 ['Arroz', 'Feijão'],
 ['Arroz']]

Executando o algoritmo apriori e salvando as regras obtidas.

In [32]:
MIN_SUPPORT = 0.3
MIN_CONFIDENCE = 0.8

rules = apriori(txs, min_support=MIN_SUPPORT, min_confidence=MIN_CONFIDENCE)
rules_list = list(rules)

for rule in rules_list: print(rule)


RelationRecord(items=frozenset({'Café', 'Manteiga'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Café'}), items_add=frozenset({'Manteiga'}), confidence=1.0, lift=2.0)])
RelationRecord(items=frozenset({'Café', 'Pão'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Café'}), items_add=frozenset({'Pão'}), confidence=1.0, lift=2.0)])
RelationRecord(items=frozenset({'Manteiga', 'Pão'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Manteiga'}), items_add=frozenset({'Pão'}), confidence=0.8, lift=1.6), OrderedStatistic(items_base=frozenset({'Pão'}), items_add=frozenset({'Manteiga'}), confidence=0.8, lift=1.6)])
RelationRecord(items=frozenset({'Café', 'Manteiga', 'Pão'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Café'}), items_add=frozenset({'Manteiga', 'Pão'}), confidence=1.0, lift=2.5), OrderedStatistic(items_base=frozenset({'Café', 'Manteiga'}), items_add=frozenset({'Pão'}), co

Ref:
- https://www.section.io/engineering-education/apriori-algorithm-in-python/
- https://splunktool.com/understanding-apyoris-output

In [38]:
Antecedente = []
Consequente = []
support = []
confidence = []
lift = []

for resultado in rules_list:
  s = resultado[1]
  result_rules = resultado[2]
  for result_rule in result_rules:
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    if 'nan' in a or 'nan' in b: continue
    if len(a) == 0 or len(b) == 0: continue
    Antecedente.append(a)
    Consequente.append(b)
    support.append(s)
    confidence.append(c)
    lift.append(l)
    final_df = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': support, 'Confidence': confidence, 'lift': lift})


In [37]:
final_df


,Antecedente,Consequente,suporte,Confidence,lift
0,[Café],[Manteiga],0.3,1.0,2.0
1,[Café],[Pão],0.3,1.0,2.0
2,[Manteiga],[Pão],0.4,0.8,1.6
3,[Pão],[Manteiga],0.4,0.8,1.6
4,[Café],"[Manteiga, Pão]",0.3,1.0,2.5
5,"[Café, Manteiga]",[Pão],0.3,1.0,2.0
6,"[Café, Pão]",[Manteiga],0.3,1.0,2.0


**Podemos ordenar estas regras por uma métrica desejada**

In [39]:
final_df.sort_values(by='lift', ascending=False)


,Antecedente,Consequente,suporte,Confidence,lift
4,[Café],"[Manteiga, Pão]",0.3,1.0,2.5
0,[Café],[Manteiga],0.3,1.0,2.0
1,[Café],[Pão],0.3,1.0,2.0
5,"[Café, Manteiga]",[Pão],0.3,1.0,2.0
6,"[Café, Pão]",[Manteiga],0.3,1.0,2.0
2,[Manteiga],[Pão],0.4,0.8,1.6
3,[Pão],[Manteiga],0.4,0.8,1.6
